# Bitácora de exploración
## Collective Intelligence - Making Recommendations

### Filtros colaborativos

Se desarrollará un sistema de recomendación basado coincidencias de un usuario con otros.

Un algorítmo de *filtrado colaborativo* usualmente funciona buscando en un grupo grande de personas y encontrando un conjunto mas pequeño con gustos similares a los de cierto usuario.

En general, filtrado colaborativo es el proceso de filtrado de información o patrones usando técnicas que involucran la colaboración entre múltiples agentes, puntos de vista, fuentes de datos, etc.

### Recolectando preferencias

Lo primero que se tiene que considerar es una representación que nos permita hablar de **personas** y sus **gustos** sobre determinados **ítems**. En este caso las personas serán críticos de películas, los gustos se representarán como las calificaciones que le asignan a películas, las cuales serán los ítems.

La representación con la que se trabajará será la de diccionarios anidados, si `d[a,b]` es un diccionario que relaciona `a` con `b`, la estructura que utilizaremos tiene la forma:

```
d[Nombre de persona, d[Nombre de película, Calificación]
```

Tanto el nombre del crítico como el nombre de la película son cadenas de caracteres; la calificación se representa como un valor numérico entre 1 y 5

In [17]:
# Un diccionario de críticos de cine con sus calificaciones de un
# conjunto pequeño de películas
critics = {
    "Lisa Rose" : {
        "Lady in the Water"  : 2.5,
        "Snakes on a Plane"  : 3.5,
        "Just My Luck"       : 3.0,
        "Superman Returns"   : 3.5,
        "You, Me and Dupree" : 2.5,
        "The Night Listener" : 3.0
    },
    "Gene Seymour" : {
        "Lady in the Water"  : 3.0,
        "Snakes on a Plane"  : 3.5,
        "Just My Luck"       : 1.5,
        "Superman Returns"   : 5.0,
        "The Night Listener" : 3.0,
        "You, Me and Dupree" : 3.5
    },
    "Michael Phillips" : {
        "Lady in the Water"  : 2.5,
        "Snakes on a Plane"  : 3.0,
        "Superman Returns"   : 3.5,
        "The Night Listener" : 4.0
    },
    "Claudia Puig" : {
        "You, Me and Dupree" : 2.5,
        "Snakes on a Plane"  : 3.5,
        "Just My Luck"       : 3.0,
        "Superman Returns"   : 4.0,
        "The Night Listener" : 4.5
    },
    "Mick LaSalle" : {
        "Lady in the Water"  : 3.0,
        "Snakes on a Plane"  : 4.0,
        "Just My Luck"       : 2.0,
        "Superman Returns"   : 3.0,
        "The Night Listener" : 3.0,
        "You, Me and Dupree" : 2.0
    },
    "Jack Matthews" : {
        "Lady in the Water"  : 3.0,
        "Snakes on a Plane"  : 4.0,
        "You, Me and Dupree" : 3.5,
        "Superman Returns"   : 5.0,
        "The Night Listener" : 3.0
    },
    "Toby" : {
        "Snakes on a Plane"  : 4.5,
        "You, Me and Dupree" : 1.0,
        "Superman Returns"   : 4.0
    },
    "Lolo" : {
        "Lili" : 1.5,
        "Lala" : 4.2
    },
    "Coco" : {
        "Cici" : 5.0,
        "Caca" : 2.4
    }
}

### Encontrando usuarios similares

Para determinar que tan similar son los gustos de dos personas se pueden utilizar varias métricas, las cuales llamamos *puntuaciones de similitud*

En este ejercicio se implementarán dos: la *distancia Euclideana* y la *correlación de Pearson*

#### Puntuación de la distancia Euclideana

Considerando $n$ películas calificadas por una cantidad de críticos, podemos encontrar que tan similar son dos personas si tomamos un espacio $n$-dimensional en donde un punto es un crítico y sus coordenadas se determinan por la calificación que le asignó dicho crítico a las películas.

La distancia Euclideana en este caso será calculada entre dos críticos considerando las películas que calificaron en común.

$$f(c^{(a)}, c^{(b)}) = \sqrt{(x^{(a)}_1 - x^{(b)}_1)^2 + \dots (x^{(a)}_n - x^{(b)}_n)^2}$$

donde $x^{(r)}_{i}$ es la calificación que le asignó el crítico $r$ a la película $i$

In [64]:
# Obtiene una lista con los ítems en común entre dos usuarios
def shared_items(data, user_a, user_b):
    return [item for item in data[user_a] if item in data[user_b]]

In [65]:
shared_items(critics, "Gene Seymour", "Lisa Rose")

['Lady in the Water',
 'Snakes on a Plane',
 'Just My Luck',
 'Superman Returns',
 'You, Me and Dupree',
 'The Night Listener']

In [68]:
# Calcula la distancia euclideana entre dos usuarios en la base de datos
def euclidean_distance(data, user_a, user_b):
    return sum([pow(data[user_a][item] - data[user_b][item], 2)
               for item in shared_items(data, user_a, user_b)])

In [69]:
euclidean_distance(critics, "Gene Seymour", "Lisa Rose")

5.75

In [72]:
# Calcula similaridad = 1/(1+distancia)
def euclidean_similarity(data, user_a, user_b):
    return 1/(1+euclidean_distance(data, user_a, user_b))

In [73]:
euclidean_similarity(critics, "Gene Seymour", "Lisa Rose")

0.14814814814814814

#### Puntuación de la correlación de Pearson

### Recomendando ítems

### Coincidiendo productos

### Construyendo un recomendador de link de del.icio.us

### Filtrado basado en ítems

### Usando el conjunto de datos de MovieLens